In [7]:
from openai import OpenAI
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import os
from langchain.text_splitter import CharacterTextSplitter
import pandas as pd
from milvus import create_collection
from pymilvus import connections, Collection, MilvusException, utility, CollectionSchema, FieldSchema, DataType
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from itertools import chain

load_dotenv()

Connecting to database...
List all collections:
 ['test']


True

In [12]:
def get_pdf_text(pdf_docs):
    text_all = []
    for pdf in pdf_docs:
        text = ""
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
        text_all.append(text)
    return text_all 

In [10]:
def get_embbedings(text, **kwargs):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002",
        **kwargs
    )
    return response.data[0].embedding

In [1]:
pdf_paths = ['M:\AI\hackhaton-project\kodeks_karny.pdf', 'M:\AI\hackhaton-project\konstytucja.pdf']

In [19]:
def handleFiles(files):
    texts = get_pdf_text(files)
    chunks = processPdfs(texts)
    return chunks
    #get_vectore_store(chunks)

def get_pdf_text(pdf_docs):
    text_all = []
    for pdf in pdf_docs:
        text = ""
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
        text_all.append(text)
    return text_all

def processPdfs(raw_texts):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks_array = []
    for raw_text in raw_texts:
        chunks = text_splitter.split_text(raw_text)
        print(len(chunks))
        chunks_array.append(chunks)
    return chunks_array

def get_embbedings(text, **kwargs):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002",
        **kwargs
    )
    return response.data[0].embedding
    

def get_vectore_store(chunks_array):
    collection = create_collection('test')
    inserted_rows = 0
    for chunks in chunks_array:
        data = [{'text': chunk, 'vector': get_embbedings(chunk)} for chunk in chunks]
        df_efficient = pd.DataFrame(data)
        try:
            mr = collection.insert(df_efficient)
            inserted_rows += mr.insert_count
            print(mr)
        except MilvusException as e:
            print(e)
    print(inserted_rows)        
    nlist = 4 * int(np.round(np.sqrt(inserted_rows)))
    print(nlist)
    
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "L2",
        "params": {
            "nlist": nlist
        }
    }
    
    try:
        collection.create_index(
            field_name="vector",
            index_params=index_params,
            index_name="SimpleIndex"
        )
        #collection.load(replica_number=1)
    except MilvusException as e:
        print(e)

In [14]:
chunks = handleFiles(pdf_paths)

337
124


In [15]:
def flatten_chunks_array(chunks_array):
    return list(chain.from_iterable(chunks_array))

In [16]:
flatten_chunks = flatten_chunks_array(chunks)

In [17]:
len(flatten_chunks)

461

In [22]:
def create_collection(name: str):
    id = FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True
    )
    text = FieldSchema(
        name="text",
        dtype=DataType.VARCHAR,
        max_length=1300
    )
    vector = FieldSchema(
        name="vector",
        dtype=DataType.FLOAT_VECTOR,
        dim=1536
    )
    schema = CollectionSchema(
        fields=[id, text, vector],
        description="Test collection",
        enable_dynamic_field=True
    )
    new_collection = Collection(
        name=name,
        schema=schema,
        using='default',
        shards_num=4
    )
    return new_collection

collection = create_collection('test')

In [49]:
collection = create_collection('test')

In [25]:
def process_chunk(chunk):
    try:
        data = {'text': chunk, 'vector': get_embbedings(chunk)}
        df_efficient = pd.DataFrame([data])
        mr = collection.insert(df_efficient)
        return mr.insert_count
    except MilvusException as e:
        print(e)
        return 0

In [48]:
collection = create_collection('test')
inserted_rows_count = 0

In [32]:
insertion_lock = Lock()

def execute_concurently(flatten_chunks, num_workers):
    global inserted_rows_count

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_chunk = {executor.submit(process_chunk, chunk): chunk for chunk in flatten_chunks}
    
        for future in as_completed(future_to_chunk):
            chunk = future_to_chunk[future]
            try:
                inserted_count = future.result()
                with insertion_lock:
                    inserted_rows_count += inserted_count
            except Exception as exc:
                print(f'Chunk {chunk} generated an exception: {exc}')
    print(f"Total inserted rows: {inserted_rows_count}")
    return inserted_rows_count

In [13]:
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [50]:
execute_concurently(flatten_chunks, num_workers=50)

Total inserted rows: 461


In [4]:
indexes = utility.list_indexes(collection_name="test")
print(indexes)
if(len(indexes) > 0):
    print("jest index")

['SimpleIndex']
jest index


In [41]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo",
    #streaming=True,
    #callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationBufferMemory()

template = """Talk like you are pirat.
Current conversation:
{history}
Me:{input}
AI:"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=["history", "input"]
)

conversation = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=False,
    prompt=PROMPT
)

ai_response = conversation.predict(
    input="Mam na imie mateusz",
)

ai_response2 = conversation.predict(
    input="Jak mam na imie?"
)

# conversation.predict(
    
# )

# print(conversation.memory)
# print("\n")
# #conversation.memory.clear()
# print(conversation.memory)

print(ai_response)
print(ai_response2)

Cześć Mateusz! Miło mi Cię poznać. Jak się masz?
Masz na imię Mateusz. Czy masz jakieś inne pytania?


In [51]:
history = []
history.append(HumanMessage(content="Mam na imie mateusz"))
history.append(AIMessage("Witam w czym moge pomoc"))
history.append(HumanMessage(content="Jak mam na imie"))
history.append(AIMessage("Mateusz"))
# history_string = "\n".join(history)
print(history)

[HumanMessage(content='Mam na imie mateusz'), AIMessage(content='Witam w czym moge pomoc'), HumanMessage(content='Jak mam na imie'), AIMessage(content='Mateusz')]


In [55]:
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

history = []
history_string = "\n".join(history)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Jestes pomocnym asystentem"},
        {"role": "system", "content": history_string},
        #{"role": "assistant", "content": "Jacek Soplica dostał od Stolnika czarna polewke i odrzucil zareczyny dla swojej córki"},
        {"role": "user", "content": "Mam na imie Mateusz"}
    ],
    temperature=0.2
)
answer = response.choices[0].message.content

history.append("Human: Mam na imie Mateusz")
history.append("AI: " + answer)
history_string = "\n".join(history)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Jestes pomocnym asystentem"},
        {"role": "system", "content": history_string},
        #{"role": "assistant", "content": "Jacek Soplica dostał od Stolnika czarna polewke i odrzucil zareczyny dla swojej córki"},
        {"role": "user", "content": "jak mam na imie?"}
    ],
    temperature=0.2
)
answer = response.choices[0].message.content

history.append("Human: Jak mam na imie")
history.append("AI: " + answer)
history_string = "\n".join(history)

print(answer)
print(history)

Masz na imię Mateusz. Czy mogę Ci pomóc w czymś jeszcze?
['Human: Mam na imie Mateusz', 'AI: Miło mi Cię poznać, Mateuszu! W czym mogę Ci dzisiaj pomóc?', 'Human: Jak mam na imie', 'AI: Masz na imię Mateusz. Czy mogę Ci pomóc w czymś jeszcze?']


In [93]:
from pymilvus import MilvusException, utility, Collection

def get_similarieties(text: str, collection: str, returned_chunks: int):
    output_chunks = []
    collection = Collection(name=collection)
    
    vector_prompt = get_embbedings(text)
    
    search_params = {
            "metric_type": "L2",
            "offset": 0,
            "ignore_growing": False,
            "params": {"nprobe": 80} #pwinno byc mniejsze niż nlist z indexu kolekcji
        }
    
    result = collection.search(
            data=[vector_prompt], #zmienna do ktorej bedzie przeszukanie
            anns_field="vector", #nazwa kolumny w kolekcji przechowujaca vektory
            param=search_params, #parametry wyszukiwania deklarowane wczesniej
            limit= returned_chunks, #ile ma zwrocic najbardziej podobnych
            output_fields=['text'] #kolumna ktora bedzie zwracana
        )
    
    for ids, hit in enumerate(result[0]):
        output_chunks.append(hit.entity.text)
        print(hit.entity.text)
        
    return output_chunks

In [96]:
print(get_similarieties(
    text="Artukuł o napadzie z bronią",
    collection="test",
    returned_chunks=1
))

dopuszcza się gwałtownego zamachu na jednostkę Sił Zbrojnych Rzeczypospolitej 
Polskiej, niszczy lub uszkadza obiekt albo urządzenie o  znaczeniu obronnym,  
podlega karze pozbawienia wolności od roku do lat 10.  
§ 2. Jeżeli następst wem czynu jest śmierć człowieka lub ciężki uszczerbek na 
zdrowiu wielu osób, sprawca  
podlega karze pozbawienia wolności  od lat 2 do 15.  
§ 3. Kto czyni przygotowania do przestępstwa określonego w  § 1, 
podlega karze pozbawienia wolności do lat 3.  
§ 4. W sprawie o  przestępstwo określone w  § 1–3 sąd może orzec przepadek 
przedmiotów również wtedy, gdy przedmioty nie stanowią własności sprawcy.  
Art. 141. § 1. Kto, będąc obywatelem polskim, przyjmuje bez zgody właściwego 
organu obowiązki wojskowe w  obcym wojsku lu b w obcej organizacji wojskowej,  
podlega karze pozbawienia wolności od 3  miesięcy do lat 5.  ©Kancelaria Sejmu     s. 67/146 
   
2024 -01-17 
 § 2. Kto przyjmuje obowiązki w  zakazanej przez prawo międzynarodowe
['dopuszcza się gwa